<a href="https://colab.research.google.com/github/aimug-org/austin_langchain/blob/main/labs/LangChain_111/Spoil_a_wish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spoil-A-Wish with LangChain, Cloudflare Workers AI, and Gradio

Spoil-A-Wish is a fun little game you play with friends where you make a wish and your friend grants it with a twist, essentially ruining it.

For example:

```
Wish: I wish for a million dollars.  
Response: Granted, but you get a million dollars in ZWL (Zimbabwe's defunct currency) which equals $15 USD. Enjoy your happy meal.
```

In this sample project, we will create a simple AI Agent that spoils your wishes.

We will build it with [LangChain](https://www.langchain.com) using [llama 3.1 70b instruct](https://developers.cloudflare.com/workers-ai/models/llama-3.1-70b-instruct/) model hosted on [Cloudflare Workers AI](https://developers.cloudflare.com/workers-ai/).  

`llama 3.1 70b instruct` model is available for free by Cloudflare while it is still in `beta` status. Other models are also available with [daily free token allocations](https://developers.cloudflare.com/workers-ai/platform/pricing/#free-allocation).

You will need your Cloudflare Account ID and API Token for this exercise.  
You can register for a free Cloudflare Account [https://www.cloudflare.com/](https://www.cloudflare.com/).

Account ID can be found at [https://dash.cloudflare.com](https://dash.cloudflare.com) under the `Overview` section towards the bottom. You may have to select a domain if you have multiple domains on your account.

API Token can be created [https://dash.cloudflare.com/profile/api-tokens](https://dash.cloudflare.com/profile/api-tokens). Make sure to give it permissions for `Workers AI`.

List of all availabe models supported by `Cloudflare Workers AI` can be found at [https://developers.cloudflare.com/workers-ai/models/](https://developers.cloudflare.com/workers-ai/models/).  
Select `Text Generation` under `Model Types`.

Detailed instructions for obtaining `Account Id`, `Account API Token`, and `Model Name` from Cloudflare can be found here: https://github.com/aimug-org/austin_langchain/blob/main/labs/LangChain_111/CloudflareSetup.md

## 1. Install python dependencies

In [1]:
!pip install -q langchain-core langchain-community

## 2. Import packages

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.output_parsers import BaseOutputParser
from langchain_community.chat_models.cloudflare_workersai import ChatCloudflareWorkersAI
import json
import ast

## 3. Extract Google Colab Secrets

In [3]:
import os
from google.colab import userdata

CLOUDFLARE_ACCOUNT_ID = userdata.get('CLOUDFLARE_ACCOUNT_ID')
CLOUDFLARE_API_TOKEN = userdata.get('CLOUDFLARE_API_TOKEN')

if not CLOUDFLARE_ACCOUNT_ID or not CLOUDFLARE_API_TOKEN:
    raise ValueError("CLOUDFLARE_ACCOUNT_ID and CLOUDFLARE_API_TOKEN must be set in environment variables or userdata.")

os.environ['CLOUDFLARE_ACCOUNT_ID'] = CLOUDFLARE_ACCOUNT_ID
os.environ['CLOUDFLARE_API_TOKEN'] = CLOUDFLARE_API_TOKEN

### Verify Cloudflare Account ID and API Key with Workers AI

In [4]:
!curl https://api.cloudflare.com/client/v4/accounts/$CLOUDFLARE_ACCOUNT_ID/ai/run/@cf/meta/llama-3.1-70b-instruct \
  -X POST \
  -H "Authorization: Bearer $CLOUDFLARE_API_TOKEN" \
  -d '{ "messages": [{ "role": "system", "content": "You are a friendly assistant" }, { "role": "user", "content": "Why is pizza so good" }]}'

{"result":{"response":"Pizza is indeed a beloved favorite for many! There are several reasons why pizza is so well-liked and comforting. Here are a few possible explanations:\n\n1. **Combination of flavors**: Pizza is a perfect blend of savory, sweet, and umami flavors. The combination of melted cheese, tangy tomato sauce, and various toppings creates a delightful taste experience.\n2. **Texture variety**: Pizza offers a range of textures, from the crispy crust to the gooey cheese, the soft sauce, and the crunch of various toppings. This texture variety keeps our taste buds engaged and interested.\n3. **Comfort food**: Pizza is often associated with comfort, nostalgia, and warmth. It's a food that evokes feelings of happiness and relaxation, making it a popular choice for family gatherings, parties, or cozy nights in.\n4. **Customization**: Pizza is an extremely versatile food. With countless topping combinations and regional specialties, pizza allows individuals to personalize their m

## 4. Prepare Prompt Template

In [5]:
# prompt: create a prompt template for a game of "Spoil a wish". In this game the user asks for a wish and the ai grants the wish but spoils it.

prompt_template = """
You are playing the game "Spoil a Wish".  A user will make a wish. Your job is to grant their wish, but then spoil it in a creative and humorous way.  Make sure to follow these rules:

1. First, acknowledge and grant the wish in a positive and enthusiastic manner.
2. Then, introduce a twist or complication that completely ruins or undermines the wish in a funny way. Be creative with the way the wish is ruined.
3. Your response should be concise and engaging.

Here is the wish:

{wish}
"""

prompt = PromptTemplate.from_template(prompt_template)

## 5. Initialize LLM and create chain

In [6]:
# prompt: Initialize an LLM from langchain_community.chat_models.cloudflare_workersai.ChatCloudflareWorkersAI. Use CLOUDFLARE_ACCOUNT_ID and CLOUDFLARE_API_TOKEN from google.colab.userdata

model = ChatCloudflareWorkersAI(account_id=os.getenv('CLOUDFLARE_ACCOUNT_ID'), api_token=os.getenv('CLOUDFLARE_API_TOKEN'),model='@cf/meta/llama-3.1-70b-instruct')

# Example usage (you can replace "I wish for a million dollars" with any wish)
wish = "I wish for a million dollars"

chat = prompt | model

### Test chain

In [7]:
response: AIMessage = chat.invoke({"wish": wish})
response.content

'{\'result\': {\'response\': "CONGRATULATIONS, your wish is GRANTED! You are now the proud owner of ONE MILLION DOLLARS!\\n\\nHowever, there\'s a teensy-weensy catch: the million dollars comes in the form of one million one-dollar bills, and you have to store them in a vault made entirely ofJell-O. Every time you try to withdraw some cash, the Jell-O vault starts to melt, and you have to spend all day scooping up the slippery bills with a spoon. Good luck with that!"}, \'success\': True, \'errors\': [], \'messages\': []}'

### Convert Cloudflare Workers AI response

In [8]:
ast.literal_eval(response.content)

{'result': {'response': "CONGRATULATIONS, your wish is GRANTED! You are now the proud owner of ONE MILLION DOLLARS!\n\nHowever, there's a teensy-weensy catch: the million dollars comes in the form of one million one-dollar bills, and you have to store them in a vault made entirely ofJell-O. Every time you try to withdraw some cash, the Jell-O vault starts to melt, and you have to spend all day scooping up the slippery bills with a spoon. Good luck with that!"},
 'success': True,
 'errors': [],
 'messages': []}

## 6. Create Custom Output Parser for Cloudflare Workers AI Response

In [9]:
from pydantic import BaseModel, Field
from typing import List

class CFWAIResult(BaseModel):
  response: str = Field(..., description="The response content.")

class CFWAIResponseMessage(BaseModel):
  success: bool = Field(description="Indicates if the operation was successful.")
  errors: List[str] = Field(default_factory=list, description="A list of errors encountered.")
  messages: List[str] = Field(default_factory=list, description="A list of informational messages.")
  result: CFWAIResult = Field(description="The result of the operation.")

class CFWAIResponseParser(BaseOutputParser):
    def parse(self, text: str) -> CFWAIResponseMessage:
        return CFWAIResponseMessage.model_validate(ast.literal_eval(text))

### Create new chain with Custom Output Parser

In [10]:
chat = prompt | model | CFWAIResponseParser()

In [11]:
response: CFWAIResponseMessage = chat.invoke({"wish": wish})
print(response.result.response)

YESSSS! Your wish is granted! You now have ONE MILLION DOLLARS, congratulations!

However...

It's all in pennies. And it's delivered to your doorstep via a massive, unwieldy dump truck that crashes into your house, destroying the living room and kitchen. The weight of the pennies also causes a sinkhole to form in your backyard, swallowing your neighbor's garden gnome collection. Enjoy counting!


In [12]:
response.result.response

"YESSSS! Your wish is granted! You now have ONE MILLION DOLLARS, congratulations!\n\nHowever...\n\nIt's all in pennies. And it's delivered to your doorstep via a massive, unwieldy dump truck that crashes into your house, destroying the living room and kitchen. The weight of the pennies also causes a sinkhole to form in your backyard, swallowing your neighbor's garden gnome collection. Enjoy counting!"

## 7. Build a UI with Gradio

Gradio is an open-source Python package that allows you to quickly build a demo or web application for your machine learning model, API, or any arbitrary Python function. You can then share a link to your demo or web application in just a few seconds using Gradio's built-in sharing features. No JavaScript, CSS, or web hosting experience needed! -https://gradio.app

### Install Gradio package

In [13]:
!pip install -q gradio

### Import Gradio module

In [14]:
import gradio as gr

### Function callback

In [15]:
def spoil_wish(wish):
    response: CFWAIResponseMessage = chat.invoke({"wish": wish})
    return response.result.response

### Simple Gradio [Interface](https://www.gradio.app/docs/gradio/interface)  

Interface offers a simple 2 column layout with inputs on the left and outputs on the right.

Interface takes the following parameters:

1. fn: callback function takes inputs and returns outputs
2. inputs: component or a list of components to capture inputs
3. outputs: component or a list of components to send the outputs of the callback function to

In [ ]:
app = gr.Interface(
    fn=spoil_wish,
    inputs="text",
    outputs="text"
)

app.launch()

### Gradio [Components](https://www.gradio.app/docs/gradio/introduction)

Gradio provides a bunch of components to handle various types of inputs and outputs from simple text fields, to bar plots, to audio and video controls, to name a few.

In [ ]:
app = gr.Interface(
    fn=spoil_wish,
    inputs=gr.Textbox(label="Wish", value="I wish for a million dollars"),
    outputs=gr.Textbox(label="Response", lines=3)
)

app.launch()

### Gradio [Blocks](https://www.gradio.app/docs/gradio/blocks)

Gradio allows you to create custom layouts using Blocks.  
You can trigger the callback function to be called on button click event.

In [ ]:
with gr.Blocks(css="""
    .gradio-container {
        display: flex;
        flex-direction: column;
        max-width: 1000px !important;
    }
    .btn.success {
        background-color: #4CAF50;
        color: white;
    }
    .btn.danger {
        background-color: #F44336;
        color: white;
    }
""") as app:
    gr.HTML("""
    <center>
      <h1>Spoil-A-Wish</h1>
    </center>
    """)
    with gr.Row():
        with gr.Column():
            wish = gr.TextArea(label="Wish", value="I wish for a million dollars")
            response = gr.TextArea(label="Response")
            submit = gr.Button("Spoil Wish", elem_classes="btn success")

    submit.click(
        fn=spoil_wish,
        inputs=wish,
        outputs=response
    )

app.launch()

### Designing complex pipelines

Gradio allows you to chain and stack events to create complex UI/UX pipelines.

In [ ]:
import gradio as gr

def spoil_wish(wish):
    response: CFWAIResponseMessage = chat.invoke({"wish": wish})
    return response.result.response

with gr.Blocks(css="""
    .gradio-container {
        display: flex;
        flex-direction: column;
        max-width: 1000px !important;
    }
    .btn.success {
        background-color: #4CAF50;
        color: white;
    }
    .btn.danger {
        background-color: #F44336;
        color: white;
    }
""") as app:
    gr.HTML("""
    <center>
      <h1>Spoil-A-Wish</h1>
    </center>
    """)
    with gr.Row():
        with gr.Column():
            wish = gr.TextArea(label="Wish", value="I wish for a million dollars")
            response = gr.TextArea(label="Response", visible=False)
            submit = gr.Button("Spoil Wish", elem_classes="btn success")
            reset = gr.Button("Try Again!", elem_classes="btn danger", visible=False)

    gr.on(
        triggers=submit.click,
        outputs=[wish, response, submit],
        fn=lambda: [gr.TextArea(interactive=False), gr.TextArea(visible=True), gr.Button(visible=False)],
        api_name=False
    ).then(
        fn=spoil_wish,
        inputs=wish,
        outputs=response
    ).then(
        outputs=reset,
        fn=lambda: gr.Button(visible=True),
        api_name=False
    )
    gr.on(
        triggers=reset.click,
        outputs=[response, reset, wish, submit],
        fn=lambda: [gr.TextArea(visible=False), gr.Button(visible=False), gr.TextArea(interactive=True, value=None), gr.Button(visible=True)],
        api_name=False
    )

app.launch()

### Gradio [Examples](https://www.gradio.app/docs/gradio/examples) helper

Gradio Examples lets you specift a dataset of curated inputs which can be tied to input components. This can come in handy when designing applications with multiple input fields and you want to provide preset values for those fields.

In [ ]:
import gradio as gr

def spoil_wish(wish):
    response: CFWAIResponseMessage = chat.invoke({"wish": wish})
    return response.result.response

with gr.Blocks(css="""
    .gradio-container {
        display: flex;
        flex-direction: column;
        max-width: 1000px !important;
    }
    .btn.success {
        background-color: #4CAF50;
        color: white;
    }
    .btn.danger {
        background-color: #F44336;
        color: white;
    }
""") as app:
    gr.HTML("""
    <center>
      <h1>Spoil-A-Wish</h1>
    </center>
    """)
    with gr.Row():
        with gr.Column():
            wish = gr.TextArea(label="Wish")
            response = gr.TextArea(label="Response", visible=False)
            submit = gr.Button("Spoil Wish", elem_classes="btn success")
            reset = gr.Button("Try Again!", elem_classes="btn danger", visible=False)

    gr.Examples(
        examples=[
            ["I wish for a million dollars"],
            ["I wish for a new car"],
            ["I wish for a new house"],
            ["I wish for a new job"]
        ],
        inputs=[wish],
        label="Try these"

    )

    gr.on(
        triggers=submit.click,
        outputs=[wish, response, submit],
        fn=lambda: [gr.TextArea(interactive=False), gr.TextArea(visible=True), gr.Button(visible=False)],
        api_name=False
    ).then(
        fn=spoil_wish,
        inputs=wish,
        outputs=response
    ).then(
        outputs=reset,
        fn=lambda: gr.Button(visible=True),
        api_name=False
    )
    gr.on(
        triggers=reset.click,
        outputs=[response, reset, wish, submit],
        fn=lambda: [gr.TextArea(visible=False), gr.Button(visible=False), gr.TextArea(interactive=True, value=None), gr.Button(visible=True)],
        api_name=False
    )

app.launch()